# 18.335 Problem Set 1
Simon Opsahl (sopsahl@mit.edu)

March 5, 2025

# Problem 0: Pset Honor Code

I will not look at 18.335 pset solutions from previous semesters. I may discuss problems with my classmates or others, but I will write up my solutions on my own. 

# Problem 1: Floating point

Non-zero floating point numbers $\mathbb{F}$ can be written as
$$ x= \pm (\frac{m}{\beta ^ t})\beta ^e$$ 
with $\beta \ge 2$ and $\beta^{-1} \le \frac{m}{\beta ^t} < 1$. $e$ is an unrestricted integer.

## part (a)

In order to identify the smallest postive integer $n$ that is not represented by $\mathbb{F}$, we must first identify that for integers $\in [0, \beta^t)$, we have a direct mapping of the mantissa to integer value because there are $t$ digits of precision. For the value $\beta^t$, we can trivially represent it as a power of $\beta$. For the case of $\beta^t + 1$, however, we must have the assumed digit as $1$, and then $t$ additional digits to represent the $1$ offset. As this requires $t+1$ digits of precision, but our mantissa has a precision of $t$, $\beta^t + 1$ is the smallest positive integer not in $\mathbb{F}$. 

For fp32, $t=24$ and thus this value is $2^{24} + 1$. The result can be verified by the code below.

In [1]:
beta = big(2.0)
t = big(24)

smallest_unsupported_value = Float32(beta^t + 1) 
one_below = Float32(beta^t)
two_below = Float32(beta^t - 1)
smallest_unsupported_value == one_below, one_below==two_below

(true, false)

For fp64, $t=53$ and thus this value is $2^{53} +1$. The result can be verified by the code below.

In [3]:
beta = big(2.0)
t = big(53)

smallest_unsupported_value = Float64(beta^t + 1)
one_below = Float64(beta^t)
two_below = Float64(beta^t - 1)
smallest_unsupported_value == one_below, one_below==two_below

(true, false)

## part (b)


In base 2, we can prove that for two floating point numbers $a \leq b$, the following inequality holds:
$$a \leq fl(\frac{a+b}{2}) \leq b$$

This proof relies on the fact that $a \le b \implies fl(a) \leq fl(b)$. We can first construct the following: 
$$ a \le b$$
$$ a + b \le b + b = 2\cdot b$$
$$ fl(a+b) \le fl(2 \cdot b)$$
$$ \frac{fl(a+b)}{2} \le \frac{fl(2 \cdot b)}{2}$$
$$ fl(\frac{fl(a+b)}{2}) \le fl(\frac{fl(2 \cdot b)}{2})$$

Because we are doing operations in base-2, the effect of mutiplying by 2 is increasing the exponent by 1, and the effect of dividing is decreasing the exponent by 1. As a result, $fl(\frac{fl(2 \cdot b)}{2}) = b$. The same logic can be used to prove that $a \le fl(\frac{a+b}{2})$. As a result, in base-2, the following is always true if $a\le b$:
$$a \leq fl(\frac{a+b}{2}) \leq b$$

In [147]:
a = 6.1111
b = 6.1112

round(round(a+b, sigdigits=5)/2, sigdigits =5) >= round(a, sigdigits=5)

false

# Problem 3: Matrix norm

## part (a)

We aim to prove that $\|A\|_\infty = \underset{1\leq i\leq m}{\max} \|a_i^\ast\|_1$. In order to do so, we can use the property of subordinate norms described in lecture that given $x \in \mathbb{C}^n$, we have $\|A\|_\beta = \max\limits_{\| x \|_\beta = 1}\|Ax\|_\beta$. In this problem, $\beta = \infty$, and thus we find $$\|A\|_\infty = \max\limits_{\| x \|_\infty = 1}\|Ax\|_\infty = \max\limits_{\| x \|_\infty = 1} (\max\limits_{1\le i \le m}|a_i^\ast x|)$$ 

We also know that $\| x \|_\infty = \max\limits_{1\le j\le n} |x_j|$, and thus $|x_j| \le 1$.  As a result, we can rewrite our norm as $$\|Ax\|_\infty =  \max\limits_{1\le i \le m}|a_i^\ast x| \le \max\limits_{1\le j\le n} \sum_{j = 1}^n|a_{i, j}| |x_j|  \le \max\limits_{1\le j\le n} \sum_{j = 1}^n|a_{i, j}| =  \max\limits_{1\le j\le n} \| a_{i}^\ast \|_1 $$

By choosing $x = sign(a_i^\ast)$, we attain this bound, and thus $\|A\|_\infty = \underset{1\leq i\leq m}{\max} \|a_i^\ast\|_1$.

In [14]:
using LinearAlgebra

# Generate a random A 
m = rand(1:100)
n = rand(1:100)
A = rand(m, n)

# Calculate the row_sums
row_sums = sum(abs.(A), dims=2)

maximum(row_sums) == opnorm(A, Inf)

true

## part (b)

The proof that $I + B$ is nonsingular relies on the lemma from lecture that the spectral radius is bounded by the norm. Thus, for $1\le k\le n$, $| \lambda_k | \le \|B\| <1$, and thus $|\lambda_k + 1| \ne 0$. As a result, the corresponding matrix is nonsingular. 

In order to identify that $\| (I + B)^{-1}\| \le \frac{1}{1-\|B\|}$, we must first remark that $B$ is small. Thus, we can expand the inversion with the Neumann series. $$(I + B)^{-1} = \sum_{k = 0}^\infty (-B)^k$$ If we take the norm of each side, then we get 
$$\|(I + B)^{-1} \|= \|\sum_{k = 0}^\infty (-B)^k\| \le \sum_{k = 0}^\infty \|(-B)^k\| \le \sum_{k = 0}^\infty \|B\|^k = \frac{1}{1- \| B\|}$$

# Problem 4: SVD

## part (a)

In this problem, we must find the 2-norm and 2-condition number of the nonsingular matrix 
$$
C = \begin{bmatrix}
\alpha I & A^\ast \\
A & \alpha I
\end{bmatrix}
$$
given that $A \in \mathbb{C}^{nxn}$ with singular values $\sigma_1 \ge ...\ge \sigma_n > 0$. From lecture, we know that $\|C\|_2$ is equivalent to the largest singular value of $C$, and $\kappa_2(C)$ is the fraction of the largest singular value over the smallest.

As a result, all that is left is to find the singular values of $C$. They can be found from the square root of the eigenvalues of $C^\ast C$. 
$$
M = C^\ast C = \begin{bmatrix}
\alpha^2 I + A^\ast A & 2\alpha A^\ast \\
2\alpha A & \alpha^2 I + AA^\ast
\end{bmatrix}
$$

As the eigenvalues of $A^\ast A$ are equivalent to $\Sigma^2$, then we can compute the eigenvalues of $M$ with the following formula: 

$$
\lambda_i^{+/-} = \frac{2\alpha^2 + 2 \sigma_i^2 \pm \sqrt{(2\alpha^2 + 2\sigma_i^2)^2 -4(\alpha^4 + \sigma_i^4 -2\alpha^2 \sigma_i^2)}}{2} = \alpha^2 + \sigma_i^2 \pm 2\alpha \sigma_i = (\alpha \pm \sigma_i)^2
$$

The singular values of $C$ are the square root of the eigenvalues of $M$, and thus we have 

$$ \mu_i^{+/-} = |\alpha \pm \sigma_i|$$


In [82]:
using LinearAlgebra

n = 5
A = rand(n, n)*10

alpha = rand()*10
while any(abs.(svd(A).S .- alpha) .< 1e-10)  
    alpha = rand()*10  
end


I_ = I(n)

largest = maximum(vcat(alpha .+ svd(A).S, abs.(alpha .- svd(A).S)))
smallest = minimum(vcat(alpha .+ svd(A).S, abs.(alpha .- svd(A).S)))
largest, largest/smallest

(39.7237754680893, 28.446865343671117)

In [83]:
I_ = I(n)
C = [alpha*I_ A'; A alpha*I_] 

opnorm(C), cond(C)

(39.72377546808932, 28.446865343671174)

## part (b)

In order to identify that $1\le \text{srank}(A) \le \text{rank}(A)$, we must first expand both the frobenius norm and the 2-norm. 
$$\|A\|_F^2 = tr(A^\ast A) = \sum_{i=1}^r \sigma_i^2$$

In addition, 
$$ \|A\|_2^2 = \sigma _1^2 $$



Because $\forall i \in (1,r], \sigma_i \le \sigma_1$, and there are $r$ singular values because rank($A^\ast A$) = rank($A$), we know that $\sum_{i=1}^r \sigma_i^2 \le r\cdot \sigma_1^2$. As a result,

$$\text{srank}(A) = \frac{\sum_{i=1}^r \sigma_i^2}{\sigma_1^2} \le r = \text{rank}(A)$$

In addition, we know trivially that $\sum_{i=1}^r \sigma_i^2 \ge \sigma_1^2$, and thus

$$\text{srank}(A) = \frac{\sum_{i=1}^r \sigma_i^2}{\sigma_1^2} \ge 1$$




In [40]:
using LinearAlgebra

function srank(A)
    A_F =  norm(A)
    A_2 = opnorm(A, 2)
    return A_F^2 / A_2^2
end

srank (generic function with 1 method)

In [44]:
m = rand(1:100)
n = rand(1:100)
A = rand(m, n)

1 <= srank(A) <= rank(A)

true